In [2]:
import re
import os
import sys
import csv
import json
import argparse
import psycopg2
import pandas as pd
from psycopg2 import sql
from datetime import datetime
from dotenv import load_dotenv
from psycopg2.extras import RealDictCursor
load_dotenv()

True

# Connect database

In [3]:
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = int(os.getenv("DB_PORT", "5432"))
DB_NAME = os.getenv("DB_NAME", "postgres")
DB_USER = os.getenv("DB_USER", "")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
CONNECT_TIMEOUT = int(os.getenv("DB_CONNECT_TIMEOUT", "10"))

In [4]:
def get_db_connection():
    """
    Trả về psycopg2 connection sử dụng biến môi trường từ .env.
    """
    conn_kwargs = {
        "host": DB_HOST,
        "port": DB_PORT,
        "dbname": DB_NAME,
        "user": DB_USER,
        "password": DB_PASSWORD,
        "connect_timeout": CONNECT_TIMEOUT,
    }
    return psycopg2.connect(cursor_factory=RealDictCursor, **conn_kwargs)

def test_connection(sql_query: str = "SELECT now() AS now"):
    """
    Thực thi 1 câu SQL test và in kết quả.
    """
    conn = None
    try:
        conn = get_db_connection()
        with conn.cursor() as cur:
            cur.execute(sql_query)
            rows = cur.fetchall()
            print(f"✓ Query executed: {sql_query}")
            for row in rows:
                print(row)
    except Exception as e:
        print(f"DB error: {e}")
    finally:
        if conn:
            conn.close()

def list_tables():
    """Liệt kê các bảng hiện có (bỏ schema hệ thống)."""
    sql_query = """
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type='BASE TABLE'
      AND table_schema NOT IN ('pg_catalog','information_schema')
    ORDER BY table_schema, table_name;
    """
    test_connection(sql_query)

def show_table_info(table_name: str, schema: str = "public", do_count: bool = False):
    """
    In thông tin bảng:
      - columns + types
      - primary key
      - indexes
      - approx size on disk
      - optional exact row count (can be slow)
    """
    conn = None
    try:
        conn = get_db_connection()
        with conn.cursor() as cur:
            # Columns + types
            cur.execute(
                """
                SELECT column_name, data_type, is_nullable, character_maximum_length
                FROM information_schema.columns
                WHERE table_schema = %s AND table_name = %s
                ORDER BY ordinal_position;
                """,
                (schema, table_name),
            )
            cols = cur.fetchall()
            if not cols:
                print(f"Table '{schema}.{table_name}' not found or has no columns.")
                return
            print(f"\nColumns for {schema}.{table_name}:")
            for c in cols:
                print(f"  - {c['column_name']}: {c['data_type']} nullable={c['is_nullable']} max_len={c['character_maximum_length']}")

            # Primary key
            cur.execute(
                """
                SELECT kcu.column_name
                FROM information_schema.table_constraints tc
                JOIN information_schema.key_column_usage kcu
                  ON tc.constraint_name = kcu.constraint_name
                 AND tc.table_schema = kcu.table_schema
                WHERE tc.table_schema = %s
                  AND tc.table_name = %s
                  AND tc.constraint_type = 'PRIMARY KEY'
                ORDER BY kcu.ordinal_position;
                """,
                (schema, table_name),
            )
            pk = [r["column_name"] for r in cur.fetchall()]
            print(f"\nPrimary key: {pk or 'NONE'}")

            # Indexes (pg_indexes)
            cur.execute(
                """
                SELECT indexname, indexdef
                FROM pg_indexes
                WHERE schemaname = %s AND tablename = %s
                ORDER BY indexname;
                """,
                (schema, table_name),
            )
            idxs = cur.fetchall()
            print(f"\nIndexes ({len(idxs)}):")
            for i in idxs:
                print(f"  - {i['indexname']}: {i['indexdef']}")

            # Size on disk (human)
            cur.execute(
                """
                SELECT
                  pg_size_pretty(pg_total_relation_size(quote_ident(%s) || '.' || quote_ident(%s))) AS total_size,
                  pg_size_pretty(pg_relation_size(quote_ident(%s) || '.' || quote_ident(%s))) AS table_size
                """,
                (schema, table_name, schema, table_name),
            )
            size_info = cur.fetchone()
            print(f"\nSize: total={size_info['total_size']} table={size_info['table_size']}")

            # Approx row estimate from pg_class
            cur.execute(
                """
                SELECT reltuples::BIGINT AS estimate_rows
                FROM pg_class c
                JOIN pg_namespace n ON n.oid = c.relnamespace
                WHERE n.nspname = %s AND c.relname = %s;
                """,
                (schema, table_name),
            )
            est = cur.fetchone()
            print(f"Estimated rows (pg_class.reltuples): {est['estimate_rows'] if est else 'N/A'}")

            # Optional exact count (use sql module to safely format identifiers)
            if do_count:
                print("\nComputing exact COUNT(*) (may be slow)...")
                q = sql.SQL("SELECT count(*) AS exact_count FROM {}.{}").format(
                    sql.Identifier(schema), sql.Identifier(table_name)
                )
                cur.execute(q)
                cnt = cur.fetchone()
                print(f"Exact rows: {cnt['exact_count']}")
    except Exception as e:
        print(f"DB error: {e}")

In [5]:
list_tables()

✓ Query executed: 
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type='BASE TABLE'
      AND table_schema NOT IN ('pg_catalog','information_schema')
    ORDER BY table_schema, table_name;
    
RealDictRow([('table_schema', 'public'), ('table_name', 'Action')])
RealDictRow([('table_schema', 'public'), ('table_name', 'Admin')])
RealDictRow([('table_schema', 'public'), ('table_name', 'Answer')])
RealDictRow([('table_schema', 'public'), ('table_name', 'AnswerTranslate')])
RealDictRow([('table_schema', 'public'), ('table_name', 'Attachment')])
RealDictRow([('table_schema', 'public'), ('table_name', 'AttachmentReference')])
RealDictRow([('table_schema', 'public'), ('table_name', 'BiometricDevice')])
RealDictRow([('table_schema', 'public'), ('table_name', 'Category')])
RealDictRow([('table_schema', 'public'), ('table_name', 'CategoryOpenTime')])
RealDictRow([('table_schema', 'public'), ('table_name', 'CategoryOpenTimeTranslate')])
RealDictRow([('table_

In [7]:
show_table_info("Poi")


Columns for public.Poi:
  - id: uuid nullable=NO max_len=None
  - created_at: timestamp without time zone nullable=NO max_len=None
  - updatedAt: timestamp without time zone nullable=NO max_len=None
  - deletedAt: timestamp without time zone nullable=YES max_len=None
  - cityId: uuid nullable=NO max_len=None
  - source: character varying nullable=NO max_len=None
  - content: text nullable=NO max_len=None
  - raw_data: jsonb nullable=YES max_len=None
  - metadata: jsonb nullable=NO max_len=None
  - placeId: character varying nullable=YES max_len=None
  - audioUrl: character varying nullable=YES max_len=None
  - location: USER-DEFINED nullable=YES max_len=None

Primary key: NONE

Indexes (1):
  - PK_101b759c3c5200a5e040a6874a0: CREATE UNIQUE INDEX "PK_101b759c3c5200a5e040a6874a0" ON public."Poi" USING btree (id)

Size: total=18 MB table=2424 kB
Estimated rows (pg_class.reltuples): 1430


In [6]:
show_table_info("PoiClean")


Columns for public.PoiClean:
  - id: uuid nullable=NO max_len=None
  - created_at: timestamp without time zone nullable=NO max_len=None
  - updatedAt: timestamp without time zone nullable=NO max_len=None
  - deletedAt: timestamp without time zone nullable=YES max_len=None
  - name: text nullable=NO max_len=None
  - lat: double precision nullable=NO max_len=None
  - address: text nullable=YES max_len=None
  - poi_type: text nullable=YES max_len=None
  - total_reviews: integer nullable=YES max_len=None
  - lon: double precision nullable=NO max_len=None
  - geom: USER-DEFINED nullable=NO max_len=None
  - stay_time: double precision nullable=YES max_len=None
  - avg_stars: double precision nullable=YES max_len=None
  - normalize_stars_reviews: double precision nullable=YES max_len=None
  - open_hours: json nullable=YES max_len=None
  - poi_type_clean: text nullable=YES max_len=None
  - main_subcategory: text nullable=YES max_len=None
  - specialization: text nullable=YES max_len=None
  - 

In [7]:
# Mở connection
conn = get_db_connection()
cur = conn.cursor()

In [8]:
cur.execute('SELECT * FROM "PoiClean"')
rows = cur.fetchall()
for poi in rows[:5]:
    print(poi)

RealDictRow([('id', '0f9d2009-9436-46a4-b354-b0261898a39e'), ('created_at', datetime.datetime(2026, 1, 7, 9, 28, 3, 360928)), ('updatedAt', datetime.datetime(2026, 1, 26, 10, 30, 11, 886556)), ('deletedAt', None), ('name', 'The Pub Coffee - Beer & Cocktail'), ('lat', 10.8294811), ('address', '18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thành phố Hồ Chí Minh, Vietnam'), ('poi_type', 'Cafe,Bar'), ('total_reviews', 181), ('lon', 106.7737852), ('geom', '0101000020E61000003F085CB285B15A40DB4C2ABFB1A82540'), ('stay_time', 25.0), ('avg_stars', 4.9), ('normalize_stars_reviews', 0.755), ('open_hours', [{'day': 'Monday', 'hours': [{'start': '00:00', 'end': '23:59'}]}, {'day': 'Tuesday', 'hours': [{'start': '00:00', 'end': '23:59'}]}, {'day': 'Wednesday', 'hours': [{'start': '00:00', 'end': '23:59'}]}, {'day': 'Thursday', 'hours': [{'start': '00:00', 'end': '23:59'}]}, {'day': 'Friday', 'hours': [{'start': '00:00', 'end': '23:59'}]}, {'day': 'Saturday', 'hours': [{'start': '00:00', 'end': '23:59'}

In [ ]:
cur.execute('SELECT * FROM "Poi"')
rows = cur.fetchall()
for poi in rows[:5]:
    print(poi)

# Các feature cần phải lấy ra từ table Poi

- id UUID PRIMARY KEY, lấy từ cột id
- name TEXT NOT NULL, lấy từ cột content
- address TEXT, lấy từ cột content 
- lat DOUBLE PRECISION NOT NULL, lấy từ cột content
- lon DOUBLE PRECISION NOT NULL, lấy từ cột content 
- geom GEOMETRY(Point, 4326) NOT NULL, tạo ra code
- poi_type TEXT, lấy từ cột content 
- stay_time DOUBLE PRECISION CHECK (stay_time >= 0), default 30
- opening_hours lấy từ cột raw_data
- crowd, children, offerings, atmosphere, highlights, popular_for, dining_options, accessibility options lấy từ metadata.additionalInfo
- avg_stars DOUBLE PRECISION CHECK (avg_stars BETWEEN 0 AND 5), lấy từ cột raw_data
- total_reviews INTEGER CHECK (total_reviews >= 0), từ cột raw_data
- normalize_stars_reviews DOUBLE PRECISION # normalize avg_stars và total_reviews về thang điểm 0-1

# Tạo 1 file csv

In [ ]:
file_path = os.path.join(os.getcwd(), "../data_csv/data.csv")
print(f"File path: {file_path}")

# Tạo thư mục nếu chưa tồn tại
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Tạo file nếu chưa tồn tại
if not os.path.exists(file_path):
    with open(file_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

# Đọc dữ liệu
if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
    df = pd.DataFrame()   # df rỗng, chưa có cột
else:
    df = pd.read_csv(file_path)

print(df)

File path: c:\Users\nguye\Desktop\vinamo\Main_Branch\Kyanon-support-localtion\scripts\get_data_from_db\../data/data.csv
Empty DataFrame
Columns: []
Index: []


# Trích xuất các trường

In [14]:
def extract_true_keys(items):
    """
    Trả về danh sách các key có giá trị True trong một list các dictionary.

    Ví dụ:
        items = [{"Wifi": True, "Outdoor seating": False}]
        → ["Wifi"]

    - Nếu items không phải list → trả về []
    - Mỗi phần tử trong items phải là dict thì mới xét
    """
    if not isinstance(items, list):
        return []
    return [
        key
        for d in items if isinstance(d, dict)
        for key, value in d.items()
        if value is True
    ]


def clean_opening_hours(data):
    """
    Làm sạch dữ liệu ngày & giờ từ Google.

    Nhiệm vụ:
    - Thay ký tự \u202f (narrow no-break space) bằng space bình thường " "
    - Giữ nguyên cấu trúc và nội dung ngày/giờ
    - Trả về danh sách cùng số lượng phần tử như input

    Input mẫu:
        [
            [{"day": "Monday", "hours": "10 AM to 5 PM"}],
            [{"day": "Tuesday", "hours": "Closed"}]
        ]

    Output:
        [
            [{"day": "Monday", "hours": "10 AM to 5 PM"}],
            [{"day": "Tuesday", "hours": "Closed"}]
        ]
    """
    cleaned = []
    for items in data:
        cleaned_items = []
        for i in items:
            cleaned_items.append({
                "day": i["day"],
                "hours": i["hours"].replace("\u202f", " ")
            })
        cleaned.append(cleaned_items)
    return cleaned

# ---- Helper: convert time string to 24h format ----
def to_24h(time_str, fallback_am_pm=None):
    time_str = time_str.strip()

    # If contains AM/PM → direct parse
    if "AM" in time_str.upper() or "PM" in time_str.upper():
        return datetime.strptime(time_str.upper(), "%I:%M %p").strftime("%H:%M") \
            if ":" in time_str else datetime.strptime(time_str.upper(), "%I %p").strftime("%H:%M")

    # If missing AM/PM, but fallback exists
    if fallback_am_pm:
        combined = f"{time_str} {fallback_am_pm}"
        try:
            return datetime.strptime(combined, "%I:%M %p").strftime("%H:%M") \
                if ":" in time_str else datetime.strptime(combined, "%I %p").strftime("%H:%M")
        except:
            pass

    # If totally unknown → assume 24h? But better skip
    return None


# ---- Main parser for hours ----
def parse_hours(hours_str):
    hours_str = hours_str.replace("\u202f", " ").strip()

    # Closed → skip
    if hours_str.lower() == "closed":
        return []

    # Open 24 hours
    if "open 24 hours" in hours_str.lower():
        return [{"start": "00:00", "end": "23:59"}]

    segments = [seg.strip() for seg in hours_str.split(",")]
    results = []

    for segment in segments:
        # Example segment: "1:30 to 4 PM"
        match = re.match(r"(.*) to (.*)", segment)
        if not match:
            continue

        start_raw, end_raw = match.groups()
        start_raw = start_raw.strip()
        end_raw = end_raw.strip()

        # Determine fallback AM/PM from end if start lacks AM/PM
        fallback = None
        if "AM" in end_raw.upper():
            fallback = "AM"
        if "PM" in end_raw.upper():
            fallback = "PM"

        start_24 = to_24h(start_raw, fallback)
        end_24 = to_24h(end_raw)

        if start_24 and end_24:
            results.append({
                "start": start_24,
                "end": end_24
            })

    return results


# ---- Main function to process full opening_hours array ----
def normalize_opening_hours(opening_hours):
    normalized = []

    for item in opening_hours:
        day = item["day"]
        hours = parse_hours(item["hours"])

        if hours:  # Skip closed days
            normalized.append({"day": day, "hours": hours})

    return normalized

def clean_opening_hours_special(data):
    cleaned_special = []
    for d in data:
        if len(d) == 0:
            cleaned_special.append([])
        else:
            item = normalize_opening_hours(d)
            cleaned_special.append(item)
    return cleaned_special


In [12]:
cur.execute('SELECT id, content, raw_data, metadata FROM "Poi"')
rows = cur.fetchall()

for row in rows[:5]:
    print(row)

RealDictRow([('id', '0f9d2009-9436-46a4-b354-b0261898a39e'), ('content', '{"name":"The Pub Coffee - Beer & Cocktail","placeId":"ChIJ7R9GGLQndTERJRircGTWzn4","description":"","lat":10.8294811,"long":106.7737852,"type":["Cafe","Bar"],"address":"18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thành phố Hồ Chí Minh, Vietnam","city":"Ho Chi Minh City","imageUrl":"pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/0d6d8596-ded7-4e10-954b-096aa2783725.jpg","photoUrls":["pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/0d6d8596-ded7-4e10-954b-096aa2783725.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/5f9da691-9b82-4ff2-86ea-7316f6f34028.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/7b15a9c4-8981-4255-be10-bd193951b4e2.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/95c293dc-86be-4fa0-b399-fed54b1f784d.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/aef8efda-45ce-4309-9785-6524c10e4d30.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/cd70d59d-945e-4942-9517-66f2d91dba5a.jpg","pois/images/ChIJ7R9GGLQndTERJRircGTWzn4/ceb00f6e-9

In [15]:
# Khởi tạo list cho từng cột
ids = []
names = []
addresses = []
lats = []
lons = []
poi_types = []
avg_stars = []
total_reviews = []

crowd = []
offerings = []
atmosphere = []
highlights = []
dining_options = []
children = []
accessibility = []
popular_for = []

opening_hours_raw = []

# ---- Xử lý toàn bộ trong 1 vòng lặp ----
for poi in rows:

    # -------------------- id --------------------
    ids.append(poi.get("id"))

    # -------------------- content --------------------
    raw = poi.get("content")
    if isinstance(raw, str):
        try:
            content = json.loads(raw)
        except:
            content = {}
    else:
        content = {}

    names.append(content.get("name"))
    addresses.append(content.get("address"))
    lats.append(content.get("lat"))
    lons.append(content.get("long"))

    # Type có thể là list hoặc string
    types = content.get("type")
    if isinstance(types, list):
        poi_types.append(",".join(types))
    else:
        poi_types.append(types)

    # -------------------- raw_data --------------------
    raw_data = poi.get("raw_data") or {}
    google = raw_data.get("google") or {}

    avg_stars.append(google.get("totalScore"))
    total_reviews.append(google.get("reviewsCount"))

    # opening_hours
    opening_hours_raw.append(google.get("openingHours"))

    # -------------------- metadata --------------------
    metadata = poi.get("metadata") or {}
    additional = metadata.get("additionalInfo") or {}

    crowd.append(extract_true_keys(additional.get("Crowd")))
    offerings.append(extract_true_keys(additional.get("Offerings")))
    atmosphere.append(extract_true_keys(additional.get("Atmosphere")))
    highlights.append(extract_true_keys(additional.get("Highlights")))
    dining_options.append(extract_true_keys(additional.get("Dining options")))
    children.append(extract_true_keys(additional.get("Children")))
    accessibility.append(extract_true_keys(additional.get("Accessibility")))
    popular_for.append(extract_true_keys(additional.get("Popular for")))

# -------------------- Fill vào DataFrame --------------------
df["id"] = ids
df["name"] = names
df["address"] = addresses
df["lat"] = lats
df["lon"] = lons
df["poi_type"] = poi_types

df["avg_stars"] = avg_stars
df["total_reviews"] = total_reviews

df["crowd"] = [", ".join(x) for x in crowd]
df["offerings"] = [", ".join(x) for x in offerings]
df["atmosphere"] = [", ".join(x) for x in atmosphere]
df["highlights"] = [", ".join(x) for x in highlights]
df["dining_options"] = [", ".join(x) for x in dining_options]
df["children"] = [", ".join(x) for x in children]
df["accessibility"] = [", ".join(x) for x in accessibility]
df["popular_for"] = [", ".join(x) for x in popular_for]

# -------------------- opening hours --------------------
result = clean_opening_hours(opening_hours_raw)
df["opening_hours"] = clean_opening_hours_special(result)


In [20]:
df.shape

(1454, 17)

In [21]:
df.head(2)

,id,name,address,lat,lon,poi_type,avg_stars,total_reviews,crowd,offerings,atmosphere,highlights,dining_options,children,accessibility,popular_for,opening_hours
0,0f9d2009-9436-46a4-b354-b0261898a39e,The Pub Coffee - Beer & Cocktail,"18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thà...",10.829481,106.773785,"Cafe,Bar",4.9,181,Groups,"Alcohol, Beer, Cocktails, Coffee, Hard liquor","Casual, Cozy","Great beer selection, Great coffee, Live music...",Table service,,,,"[{'day': 'Monday', 'hours': [{'start': '00:00'..."
1,02887955-963a-43ac-b0f7-355d7d7cfacf,Julieta,"C. Sta. Lucía, 9, Distrito Centro, 29008 Málag...",36.722011,-4.421780,Cafe,4.3,2053,"College students, Groups, Tourists","Alcohol, Beer, Coffee, Healthy options, Organi...","Casual, Cozy, Trendy","Great coffee, Great dessert, Great tea selection","Breakfast, Brunch, Lunch, Dessert, Seating, Ta...",Good for kids,"Wheelchair accessible entrance, Wheelchair acc...","Breakfast, Good for working on laptop","[{'day': 'Monday', 'hours': [{'start': '08:00'..."


# Xuất vào csv

In [22]:
df.to_csv(file_path, index=False)